In [ ]:
!pip install earthengine-api


##DESCRIPTION
We...

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Xác thực tài khoản Earth Engine
import ee
ee.Authenticate()

In [ ]:
import pandas as pd
import numpy as np

# Initialize Earth Engine
ee.Initialize(project='ee-nguyendangkhoi9517')

# Load DataFrame from CSV starting from row 418
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/gfd.csv')

# Define square size (10 km² -> 10,000 meters)
square_size_m = 10000  # each side of the square

# Prepare an empty list to store results
results = []

# Loop through each row in the DataFrame starting from row 418
for index, row in df.iterrows():
    center_lon = row['dfo_centroid_x']
    center_lat = row['dfo_centroid_y']
    area_km2 = row['gfd_area']  # Get area from column

    # Calculate the radius of the area of interest in meters
    radius = np.sqrt(area_km2) * 1000 / 2  # Radius in meters

    # Skip if radius is zero or very close to zero
    if radius <= 0:
        print(f"Skipping event {index} due to zero radius.")
        continue

    # Parse and format the event's start and end dates
    try:
        began_date_raw = row['dfo_began']
        ended_date_raw = row['dfo_ended']

        # Convert two-digit years if necessary
        if len(began_date_raw.split('/')[-1]) == 2:
            began_date_raw = began_date_raw[:-2] + '20' + began_date_raw[-2:]
        if len(ended_date_raw.split('/')[-1]) == 2:
            ended_date_raw = ended_date_raw[:-2] + '20' + ended_date_raw[-2:]

        began_date = pd.to_datetime(began_date_raw, format='%m/%d/%Y', errors='coerce')
        ended_date = pd.to_datetime(ended_date_raw, format='%m/%d/%Y', errors='coerce')

        if pd.isna(began_date) or pd.isna(ended_date):
            print(f"Invalid date format for event {index}. Skipping this event.")
            continue

        # Format dates to strings
        began_date_str = began_date.strftime('%Y-%m-%d')
        ended_date_str = ended_date.strftime('%Y-%m-%d')
    except Exception as e:
        print(f"Error parsing dates for event {index}: {e}")
        continue

    # Create the central point and circular area around it
    point = ee.Geometry.Point(center_lon, center_lat)
    circle = point.buffer(radius)

    # Define the number of squares along each axis to cover the circle
    num_squares = int(np.ceil(radius / square_size_m))

    # Generate grid squares around the central point
    squares = []
    for i in range(-num_squares, num_squares + 1):
        for j in range(-num_squares, num_squares + 1):
            # Calculate the bounds for each 10 km² square
            square = ee.Geometry.Rectangle([
                center_lon + i * square_size_m / 111320 - square_size_m / 111320 / 2,
                center_lat + j * square_size_m / 111320 - square_size_m / 111320 / 2,
                center_lon + i * square_size_m / 111320 + square_size_m / 111320 / 2,
                center_lat + j * square_size_m / 111320 + square_size_m / 111320 / 2
            ])
            # Only add squares that intersect with the circle
            if circle.intersects(square):
                squares.append(square)

    # Get precipitation data from CHIRPS dataset
    chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')

    # Calculate average precipitation for different time windows
    for square_index, square in enumerate(squares):
        try:
            # 3 days, 7 days, and 30 days precipitation calculations
            precip_3_days = chirps.filterDate(began_date - pd.Timedelta(days=3), began_date_str) \
                .mean().reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=square,
                    scale=5000,
                    maxPixels=1e9
                ).get('precipitation').getInfo()

            precip_7_days = chirps.filterDate(began_date - pd.Timedelta(days=7), began_date_str) \
                .mean().reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=square,
                    scale=5000,
                    maxPixels=1e9
                ).get('precipitation').getInfo()

            precip_30_days = chirps.filterDate(began_date - pd.Timedelta(days=30), began_date_str) \
                .mean().reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=square,
                    scale=5000,
                    maxPixels=1e9
                ).get('precipitation').getInfo()

            # Add result to list
            square_center = square.centroid().coordinates().getInfo()
            row_data = {
                'event_index': index,
                'began_date': began_date_str,
                'ended_date': ended_date_str,
                'square_center_lat': square_center[1],
                'square_center_lon': square_center[0],
                'precipitation_3_days': precip_3_days,
                'precipitation_7_days': precip_7_days,
                'precipitation_30_days': precip_30_days
            }
            results.append(row_data)
        except Exception as e:
            print(f"Failed to calculate precipitation for square {square_index} of event {index}: {e}")
            continue

# Convert results to DataFrame and save to CSV
results_df = pd.DataFrame(results)
results_df.to_csv('/content/drive/MyDrive/Colab Notebooks/precipitation_data_with_timeframes.csv', index=False)
print("Results saved to CSV!")


{'event_index': 0, 'began_date': '2000-02-18', 'ended_date': '2000-03-01', 'square_center_lat': -31.357890788404532, 'square_center_lon': 143.60796888249774, 'precipitation_3_days': 0, 'precipitation_7_days': 0.8187821259198815, 'precipitation_30_days': 0.3499156443332642}
{'event_index': 0, 'began_date': '2000-02-18', 'ended_date': '2000-03-01', 'square_center_lat': -31.268059683426088, 'square_center_lon': 143.60796888249715, 'precipitation_3_days': 0, 'precipitation_7_days': 0.8877348255783261, 'precipitation_30_days': 0.5095575333578413}
{'event_index': 0, 'began_date': '2000-02-18', 'ended_date': '2000-03-01', 'square_center_lat': -31.178228578317242, 'square_center_lon': 143.60796888249791, 'precipitation_3_days': 0, 'precipitation_7_days': 1.1457554217288912, 'precipitation_30_days': 0.4482473810962703}
{'event_index': 0, 'began_date': '2000-02-18', 'ended_date': '2000-03-01', 'square_center_lat': -31.357890788405385, 'square_center_lon': 143.69780000000085, 'precipitation_3_day

KeyboardInterrupt: 